In [1]:
import os
import shutil
import unittest
from catvae.trainer import LightningCountVAE
from catvae.sim import multinomial_bioms
from biom import Table
from biom.util import biom_open
import numpy as np
from pytorch_lightning import Trainer
import argparse
import seaborn as sns

In [2]:
import torch
torch.__version__

'1.6.0'

# Simulate data

In [3]:
np.random.seed(0)
sims = multinomial_bioms(k=4, D=30, N=250, M=10000)
Y = sims['Y']
parts = Y.shape[0] // 10
samp_ids = list(map(str, range(Y.shape[0])))
obs_ids = list(map(str, range(Y.shape[1])))
train = Table(Y[:parts * 8].T, obs_ids, samp_ids[:parts * 8])
test = Table(Y[parts * 8 : parts * 9].T,
             obs_ids, samp_ids[parts * 8 : parts * 9])
valid = Table(Y[parts * 9:].T, obs_ids, samp_ids[parts * 9:])
tree = sims
with biom_open('train.biom', 'w') as f:
    train.to_hdf5(f, 'train')
with biom_open('test.biom', 'w') as f:
    test.to_hdf5(f, 'test')
with biom_open('valid.biom', 'w') as f:
    valid.to_hdf5(f, 'valid')

In [4]:
sims['tree'].write('basis.nwk')

'basis.nwk'

# Train model

In [5]:
output_dir = 'output'
args = [
    '--train-biom', 'train.biom',
    '--test-biom', 'train.biom',
    '--val-biom', 'train.biom',
    '--basis-file', 'basis.nwk',
    '--output-directory', output_dir,
    '--epochs', '30000',
    '--batch-size', '200',
    '--num-workers', '10',
    '--scheduler', 'cosine',
    '--learning-rate', '1e-1',
    '--n-latent', '4',
    '--gpus', '1'
]
parser = argparse.ArgumentParser(add_help=False)
parser = LightningCountVAE.add_model_specific_args(parser)
parser.add_argument('--num-workers', type=int)
parser.add_argument('--gpus', type=int)
args = parser.parse_args(args)
model = LightningCountVAE(args)
model.set_eigs(sims['eigvectors'], sims['eigs'])

trainer = Trainer(
    max_epochs=args.epochs,
    gpus=args.gpus,
    check_val_every_n_epoch=10,
    # profiler=profiler,
    fast_dev_run=False,
    # auto_scale_batch_size='power'
)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
/home/juermieboop/miniconda3/envs/catvae/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name  | Type         | Params
---------------------------------------
0 | model | LinearCatVAE | 6 K   


tensor([[-6.6151e-04, -1.2041e-03,  1.8023e-03,  9.4212e-04],
        [ 1.1088e-03, -3.1966e-04,  6.2828e-04, -1.1048e-03],
        [-4.6673e-04,  1.1367e-03, -4.8688e-04,  5.9106e-04],
        [-9.5821e-04, -7.2369e-05,  2.0285e-04,  4.9363e-05],
        [-1.6317e-04, -1.0512e-04,  5.1375e-04,  6.3205e-04],
        [ 1.5434e-03, -3.0442e-04,  9.0776e-04, -8.7771e-04],
        [-2.7815e-04, -1.0713e-03, -6.0843e-04,  9.4978e-04],
        [ 1.8745e-04, -9.0518e-04, -8.7342e-04,  4.7239e-04],
        [-1.5246e-03,  1.5120e-04,  6.3772e-04,  3.1237e-04],
        [-9.2798e-04,  6.8641e-04, -1.2747e-03,  4.6764e-04],
        [ 8.1069e-04, -4.7660e-04,  5.9866e-04, -2.0696e-04],
        [-5.3539e-04,  1.7420e-03,  7.5694e-04,  7.6580e-04],
        [ 1.0877e-04, -1.6315e-03, -7.0067e-04, -7.6027e-04],
        [ 7.6635e-04,  1.2570e-05,  8.8399e-04, -2.2097e-03],
        [ 4.3665e-05,  1.2349e-03,  8.5998e-06, -9.9088e-04],
        [ 1.4860e-03,  1.0444e-03, -1.1456e-04,  7.4403e-04],
        

tensor([[-6.6151e-04, -1.2041e-03,  1.8023e-03,  9.4212e-04],
        [ 1.1088e-03, -3.1966e-04,  6.2828e-04, -1.1048e-03],
        [-4.6673e-04,  1.1367e-03, -4.8688e-04,  5.9106e-04],
        [-9.5821e-04, -7.2369e-05,  2.0285e-04,  4.9363e-05],
        [-1.6317e-04, -1.0512e-04,  5.1375e-04,  6.3205e-04],
        [ 1.5434e-03, -3.0442e-04,  9.0776e-04, -8.7771e-04],
        [-2.7815e-04, -1.0713e-03, -6.0843e-04,  9.4978e-04],
        [ 1.8745e-04, -9.0518e-04, -8.7342e-04,  4.7239e-04],
        [-1.5246e-03,  1.5120e-04,  6.3772e-04,  3.1237e-04],
        [-9.2798e-04,  6.8641e-04, -1.2747e-03,  4.6764e-04],
        [ 8.1069e-04, -4.7660e-04,  5.9866e-04, -2.0696e-04],
        [-5.3539e-04,  1.7420e-03,  7.5694e-04,  7.6580e-04],
        [ 1.0877e-04, -1.6315e-03, -7.0067e-04, -7.6027e-04],
        [ 7.6635e-04,  1.2570e-05,  8.8399e-04, -2.2097e-03],
        [ 4.3665e-05,  1.2349e-03,  8.5998e-06, -9.9088e-04],
        [ 1.4860e-03,  1.0444e-03, -1.1456e-04,  7.4403e-04],
        

RuntimeError: sparse tensors do not have strides
Exception raised from strides at /opt/conda/conda-bld/pytorch_1595629395347/work/aten/src/ATen/SparseTensorImpl.cpp:52 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x4d (0x7f3d7420877d in /home/juermieboop/miniconda3/envs/catvae/lib/python3.8/site-packages/torch/lib/libc10.so)
frame #1: at::SparseTensorImpl::strides() const + 0xb2 (0x7f3d52e88402 in /home/juermieboop/miniconda3/envs/catvae/lib/python3.8/site-packages/torch/lib/libtorch_cpu.so)
frame #2: <unknown function> + 0xb3737b (0x7f3d531f037b in /home/juermieboop/miniconda3/envs/catvae/lib/python3.8/site-packages/torch/lib/libtorch_cpu.so)
frame #3: <unknown function> + 0xb3d655 (0x7f3d531f6655 in /home/juermieboop/miniconda3/envs/catvae/lib/python3.8/site-packages/torch/lib/libtorch_cpu.so)
frame #4: <unknown function> + 0xb3da80 (0x7f3d531f6a80 in /home/juermieboop/miniconda3/envs/catvae/lib/python3.8/site-packages/torch/lib/libtorch_c

In [ ]:
a = torch.randn(2, 3).to_sparse().requires_grad_(True)
b = torch.randn(3, 2, requires_grad=True)
#y = torch.sparse.mm(a, b)
y = a @ b
y.sum().backward()  
a.grad

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir lightning_logs

In [ ]:
trainer.model

# Evaluate the model

In [ ]:
!ls lightning_logs/version_1/checkpoints

In [ ]:
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
checkpoint_dir = 'lightning_logs/version_1/checkpoints'
path = f'{checkpoint_dir}/epoch=139.ckpt'
model = LightningCountVAE.load_from_checkpoint(path).cuda()

In [ ]:
#W = model.model.get_loadings()
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist

W = model.model.decoder.weight.detach().cpu().numpy()

d_estW = pdist(W)
dW = pdist(sims['W'])
plt.scatter(dW, d_estW)
plt.plot(np.linspace(0, 4), np.linspace(0, 4), 'r')
plt.xlabel('Predicted correlations')
plt.ylabel('Actual correlations')

print(pearsonr(dW, d_estW))

In [ ]:
eigvals = (W**2).sum(axis=0)
Weig = W / np.sqrt(eigvals)

In [ ]:
Weig.T @ Weig

In [ ]:
Wu, Ws, Wv = np.linalg.svd(W)
k = W.shape[1]
Wu = Wu[:, :k]
Ws = Ws[:k]
Wv = Wv[:k, :]

In [ ]:
W.T @ W

In [ ]:
import torch
from catvae.dataset.biom import BiomDataset
from skbio.stats.composition import alr_inv, closure
valid_dataset = BiomDataset(valid)

In [ ]:
decoder_np = model.model.decoder.weight.detach().cpu().numpy()
z = [] 
pred_z = []
pred_probs = []
counts = []
for i in range(len(valid_dataset)):
    cnts, batch_idx = valid_dataset[i]
    counts.append(closure(cnts))
    cnts = torch.Tensor(cnts).cuda().unsqueeze(0)
    res = model.forward(cnts)
    z.append(sims['z'][parts * 8 + i])
    pred_z.append(res['qz_m'].cpu().detach().numpy())
    pred_probs.append(alr_inv(res['px_mean'].cpu().detach().numpy()))

In [ ]:
counts = np.vstack(counts)
pred_probs = np.vstack(pred_probs)
pred_z = np.vstack(pred_z)

In [ ]:
from scipy.spatial.distance import pdist
d_predz = pdist(pred_z)
dz = pdist(z)

In [ ]:
z = np.vstack(z)

In [ ]:
sns.heatmap(pred_z)

In [ ]:
sns.heatmap(z)

In [ ]:
#W = model.model.get_loadings()
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist

W = model.model.decoder.weight.detach().cpu().numpy()

d_estW = pdist(W)
dW = pdist(sims['W'])
plt.scatter(dW, d_estW)
print(pearsonr(dW, d_estW))

In [ ]:
lam = np.ravel(sims['z'][parts * 8 : parts * 8 + 50] @ sims['W'].T)
pred_lam = np.ravel(z @ W.T)
plt.scatter(lam, pred_lam)
print(pearsonr(lam, pred_lam))

In [ ]:
z.shape, W.shape, lam.shape, sims['z'].shape, sims['W'].shape

In [ ]:
from scipy.stats import entropy
ens = []
for i in range(counts.shape[0]):
    e = entropy(counts[i], pred_probs[i])
    ens.append(e)

In [ ]:
sns.distplot(ens, bins=20)

In [ ]:
sns.heatmap(pred_probs)

In [ ]:
sns.heatmap(counts)

In [ ]:
plt.scatter(counts.ravel()+1, pred_probs.ravel()+1)
plt.xscale('log')
plt.yscale('log')

In [ ]:
pearsonr(counts.ravel()+1, pred_probs.ravel()+1)

In [ ]:
torch.exp(model.model.encoder.variational_logvars)

In [ ]:
parser

In [ ]:
# scratch work with least squares

from gneiss.balances import _balance_basis
from gneiss.cluster import random_linkage
d = 100
basis = _balance_basis(random_linkage(d))[0]

Id = np.eye(d)
dd = (1 / d) * np.ones((d, d))

A = Id - dd
b = np.random.randn(d)

x, err, rank, s = np.linalg.lstsq(A, b, rcond=None)